In [ ]:
#default_exp explorer

In [ ]:
#export
from fastai2.vision.all import *
from fastexplorer.representation import *
from starlette.applications import Starlette
from starlette.endpoints import WebSocketEndpoint
from starlette.websockets import WebSocketDisconnect
import uvicorn, asyncio, json, logging
import nest_asyncio
nest_asyncio.apply()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
path = untar_data(URLs.MNIST_TINY)
dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False)

# Fast Explorer

> Here we will setup the server and store the model representation on the fastai `Learn` object..

In [ ]:
#export
class FastExplorer:
    'Wrapper around `Representation` and `ProxyServer`.'
    def __init__(self, learn, host='0.0.0.0', port=8000):
        store_attr(self, 'learn,host,port')
        self.representation = learn.to_representation()
        self.server = Starlette()
        self.endpoint = self.server.websocket_route('/ws')(self.endpoint)
        self.socket = None
        
    def __repr__(self): return f'{self.__class__.__name__} ()'
    
    async def endpoint(self, websocket):
        # Open socket
        await websocket.accept()
        msg = await websocket.receive_json()
        self.socket = websocket
        logging.info(f'Client connected: {msg}')
        
        # Manage requests
        await websocket.send_json({'event': 'representation_data', 'msg': self.representation.to_json()})
        
        # Close socket
#         state[client_type] = False
#         websocket_dict.pop(client_type)
        await close_client(websocket, 'Finalized by server')
        logging.info(f'Client closed: {msg}')    
    
    def serve(self): uvicorn.run(self.server, host=self.host, port=self.port)

In [ ]:
#export
async def close_client(websocket, msg):
    await websocket.send_json({'event': 'close', 'msg': msg})
    await websocket.close()

In [ ]:
#export
@patch
@delegates(FastExplorer.__init__)
def fastexplorer(self:Learner, **kwargs):
    if not hasattr(self, 'explorer'): self.explorer = FastExplorer(self, **kwargs)
    self.explorer = FastExplorer(self, **kwargs)
    self.explorer.serve()

In [ ]:
#hide
learn.fastexplorer()

INFO:     Started server process [16083]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     ('127.0.0.1', 48340) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 49390) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 49704) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 49982) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 50214) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'we

INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 58924) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 59150) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 59230) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 59268) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 59302) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'client': 'web_client'}
INFO:     Client closed: {'client': 'web_client'}
INFO:     ('127.0.0.1', 59334) - "WebSocket /ws" [accepted]
INFO:     Client conne

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_server.ipynb.
Converted 01_representation.ipynb.
Converted 02_fastexplorer.ipynb.
Converted 10_tutorial.ipynb.
Converted index.ipynb.
